In [ ]:
import glob
import os

import mne
from pymatreader import read_mat
import pandas as pd
import numpy as np
from pickle import load

import torch
import torch.nn as nn
from torch.utils.data import Dataset

from model import OneEncoderNet, TwoEncoderNet, TemporalNet, CWT2DCNN, MyPipeline

data_dir = '../../data/test'
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


CROP_LEN_ = 250

label_dict = {
    0: 'frontside_kickturn',
    1: 'backside_kickturn',
    2: 'pumping'
}

c:\Users\teraoka_r\workspace\development\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def get_data(data_dir):
    data_dict = {}

    for mat_data in glob.glob(f'{data_dir}/*.mat'):
        subject = mat_data.split('\\')[-1].split('.')[0]
        data = read_mat(mat_data)
        data_dict[subject] = data['data']

    ch_names = [c.replace(' ', '') for c in data['ch_labels']]
    diff_list = [
        #横方向
        'F3_F4',
        'FCz_FC1', 'FCz_FC2', 'FCz_FC3', 'FCz_FC4', 'FCz_FC5', 'FCz_FC6', 'FC1_FC2', 'FC3_FC4', 'FC5_FC6',
        'Cz_C1', 'Cz_C2', 'Cz_C3', 'Cz_C4', 'Cz_C5', 'Cz_C6', 'C1_C2', 'C3_C4', 'C5_C6',
        'CPz_CP1', 'CPz_CP2', 'CPz_CP3', 'CPz_CP4', 'CPz_CP5', 'CPz_CP6', 'CP1_CP2', 'CP3_CP4', 'CP5_CP6',
        'P3_P4',
        #縦方向
        'Cz_FCz', 'C1_FC1', 'C2_FC2', 'C3_FC3', 'C4_FC4', 'C5_FC5', 'C6_FC6',
        'Cz_CPz', 'C1_CP1', 'C2_CP2', 'C3_CP3', 'C4_CP4', 'C5_CP5', 'C6_CP6',
        'FCz_CPz', 'FC1_CP1', 'FC2_CP2', 'FC3_CP3', 'FC4_CP4', 'FC5_CP5', 'FC6_CP6',
    ]

    use_ch = []
    for item in diff_list:
        ch1 = item.split('_')[0]
        ch2 = item.split('_')[1]
        use_ch.append(ch1)
        use_ch.append(ch2)

    use_ch = list(set(use_ch))
    use_ch_dict = {ch_names[idx]:idx for idx in range(len(ch_names)) if ch_names[idx] in use_ch}
    
    return data_dict, diff_list, use_ch_dict


class SkateDataset(Dataset):
    def __init__(self, fold, data, diff_list, use_ch_dict):
        self.diff_list = diff_list
        self.use_ch_dict = use_ch_dict
        
        self.iqr = np.load(f'../../data/scaler/iqr{fold}.npy', allow_pickle=True)
        self.median = np.load(f'../../data/scaler/median{fold}.npy', allow_pickle=True)
        self.iqr = self.iqr.reshape(72, 1)
        self.median = self.median.reshape(72, 1)

        eeg_signals = []
        for data_ in data:
            data_ = (data_ - self.median) / self.iqr
            eeg_signal = []
            for ch_num, channels in enumerate(self.diff_list):
                ch1_name = channels.split('_')[0]
                ch2_name = channels.split('_')[1]
                ch1 = data_[self.use_ch_dict[ch1_name]]
                ch2 = data_[self.use_ch_dict[ch2_name]]
                signal = ch1 - ch2
                eeg_signal.append(signal)
            eeg_signal = np.stack(eeg_signal)
            eeg_signals.append(eeg_signal)
        self.eeg_signals = np.stack(eeg_signals)
        self.eeg_signals_72 = data

    def __len__(self):
        return len(self.eeg_signals.shape[0])

    def __getitem__(self, index):
        data = self.eeg_signals[index]
        data_72 = self.eeg_signals_72[index]
        
        return data, data_72

In [3]:
data_dict, diff_list, use_ch_dict = get_data(data_dir)

In [10]:
def get_model(fold):
    model_list_eeg = []
    model = OneEncoderNet()
    load_weights = torch.load(f'../1st_train_1D/one-encoder/model/model{fold}.pth', map_location=device, weights_only=True)
    model.load_state_dict(load_weights)
    model = model.eval().to(device)
    model_list_eeg.append(model)

    model = TwoEncoderNet()
    load_weights = torch.load(f'../1st_train_1D/two-encoder/model/model{fold}.pth', map_location=device, weights_only=True)
    model.load_state_dict(load_weights)
    model = model.eval().to(device)
    model_list_eeg.append(model)

    model = TemporalNet(model_name='tf_efficientnet_b0.in1k')
    load_weights = torch.load(f'../1st_train_1Dtemporal/tf_efficientnet_b0/model/model{fold}.pth', map_location=device, weights_only=True)
    model.load_state_dict(load_weights)
    model = model.eval().to(device)
    model_list_eeg.append(model)

    model = TemporalNet(model_name='tf_efficientnet_b3.in1k')
    load_weights = torch.load(f'../1st_train_1Dtemporal/tf_efficientnet_b3/model/model{fold}.pth', map_location=device, weights_only=True)
    model.load_state_dict(load_weights)
    model = model.eval().to(device)
    model_list_eeg.append(model)

    model = TemporalNet(model_name='efficientvit_b1.r256_in1k')
    load_weights = torch.load(f'../1st_train_1Dtemporal/efficientvit_b1/model/model{fold}.pth', map_location=device, weights_only=True)
    model.load_state_dict(load_weights)
    model = model.eval().to(device)
    model_list_eeg.append(model)

    model = TemporalNet(model_name='tf_efficientnetv2_b0.in1k')
    load_weights = torch.load(f'../1st_train_1Dtemporal/tf_efficientnetv2_b0/model/model{fold}.pth', map_location=device, weights_only=True)
    model.load_state_dict(load_weights)
    model = model.eval().to(device)
    model_list_eeg.append(model)


    model_list_cwt = []
    model = CWT2DCNN(model_name='tf_efficientnet_b0.in1k')
    load_weights = torch.load(f'../1st_train_2D/tf_efficientnet_b0/model/model{fold}.pth', map_location=device, weights_only=True)
    model.load_state_dict(load_weights)
    model = model.eval().to(device)
    model_list_cwt.append(model)

    model = CWT2DCNN(model_name='tf_efficientnet_b3.in1k')
    load_weights = torch.load(f'../1st_train_2D/tf_efficientnet_b3/model/model{fold}.pth', map_location=device, weights_only=True)
    model.load_state_dict(load_weights)
    model = model.eval().to(device)
    model_list_cwt.append(model)

    model = CWT2DCNN(model_name='efficientvit_b0.r224_in1k')
    load_weights = torch.load(f'../1st_train_2D/efficientvit_b0/model/model{fold}.pth', map_location=device, weights_only=True)
    model.load_state_dict(load_weights)
    model = model.eval().to(device)
    model_list_cwt.append(model)

    model = CWT2DCNN(model_name='efficientvit_b3.r224_in1k')
    load_weights = torch.load(f'../1st_train_2D/efficientvit_b3/model/model{fold}.pth', map_location=device, weights_only=True)
    model.load_state_dict(load_weights)
    model = model.eval().to(device)
    model_list_cwt.append(model)

    model = CWT2DCNN(model_name='tf_efficientnetv2_b0.in1k')
    load_weights = torch.load(f'../1st_train_2D/tf_efficientnetv2_b0/model/model{fold}.pth', map_location=device, weights_only=True)
    model.load_state_dict(load_weights)
    model = model.eval().to(device)
    model_list_cwt.append(model)


    model_list_72 = []
    model = OneEncoderNet(in_channels=72)
    load_weights = torch.load(f'../1st_train_1D/one-encoder_72/model/model{fold}.pth', map_location=device, weights_only=True)
    model.load_state_dict(load_weights)
    model = model.eval().to(device)
    model_list_72.append(model)

    model = TwoEncoderNet(in_channels=72)
    load_weights = torch.load(f'../1st_train_1D/two-encoder_72/model/model{fold}.pth', map_location=device, weights_only=True)
    model.load_state_dict(load_weights)
    model = model.eval().to(device)
    model_list_72.append(model)

    model = TemporalNet(model_name='tf_efficientnet_b0.in1k', in_channels=72)
    load_weights = torch.load(f'../1st_train_1Dtemporal/tf_efficientnet_b0_72/model/model{fold}.pth', map_location=device, weights_only=True)
    model.load_state_dict(load_weights)
    model = model.eval().to(device)
    model_list_72.append(model)

    model = TemporalNet(model_name='tf_efficientnet_b3.in1k', in_channels=72)
    load_weights = torch.load(f'../1st_train_1Dtemporal/tf_efficientnet_b3_72/model/model{fold}.pth', map_location=device, weights_only=True)
    model.load_state_dict(load_weights)
    model = model.eval().to(device)
    model_list_72.append(model)

    model = TemporalNet(model_name='efficientvit_b1.r256_in1k', in_channels=72)
    load_weights = torch.load(f'../1st_train_1Dtemporal/efficientvit_b1_72/model/model{fold}.pth', map_location=device, weights_only=True)
    model.load_state_dict(load_weights)
    model = model.eval().to(device)
    model_list_72.append(model)

    model_lists = []
    model_lists.append(model_list_eeg)
    model_lists.append(model_list_cwt)
    model_lists.append(model_list_72)

    return model_lists


def get_lgb_model(fold):
    model_list_eeg = []
    temp = []
    for fold_ in range(3):
        model = load(open(f'../1st_train_1D/one-encoder/lgb_model/lgb_fold{fold}_{fold_}', 'rb'))
        temp.append(model)
    model_list_eeg.append(temp)

    temp = []
    for fold_ in range(3):
        model = load(open(f'../1st_train_1D/two-encoder/lgb_model/lgb_fold{fold}_{fold_}', 'rb'))
        temp.append(model)
    model_list_eeg.append(temp)

    temp = []
    for fold_ in range(3):
        model = load(open(f'../1st_train_1Dtemporal/tf_efficientnet_b0/lgb_model/lgb_fold{fold}_{fold_}', 'rb'))
        temp.append(model)
    model_list_eeg.append(temp)

    temp = []
    for fold_ in range(3):
        model = load(open(f'../1st_train_1Dtemporal/tf_efficientnet_b3/lgb_model/lgb_fold{fold}_{fold_}', 'rb'))
        temp.append(model)
    model_list_eeg.append(temp)

    temp = []
    for fold_ in range(3):
        model = load(open(f'../1st_train_1Dtemporal/efficientvit_b1/lgb_model/lgb_fold{fold}_{fold_}', 'rb'))
        temp.append(model)
    model_list_eeg.append(temp)

    temp = []
    for fold_ in range(3):
        model = load(open(f'../1st_train_1Dtemporal/tf_efficientnetv2_b0/lgb_model/lgb_fold{fold}_{fold_}', 'rb'))
        temp.append(model)
    model_list_eeg.append(temp)

    
    model_list_cwt = []
    temp = []
    for fold_ in range(3):
        model = load(open(f'../1st_train_2D/tf_efficientnet_b0/lgb_model/lgb_fold{fold}_{fold_}', 'rb'))
        temp.append(model)
    model_list_cwt.append(temp)

    temp = []
    for fold_ in range(3):
        model = load(open(f'../1st_train_2D/tf_efficientnet_b3/lgb_model/lgb_fold{fold}_{fold_}', 'rb'))
        temp.append(model)
    model_list_cwt.append(temp)

    temp = []
    for fold_ in range(3):
        model = load(open(f'../1st_train_2D/efficientvit_b0/lgb_model/lgb_fold{fold}_{fold_}', 'rb'))
        temp.append(model)
    model_list_cwt.append(temp)

    temp = []
    for fold_ in range(3):
        model = load(open(f'../1st_train_2D/efficientvit_b3/lgb_model/lgb_fold{fold}_{fold_}', 'rb'))
        temp.append(model)
    model_list_cwt.append(temp)

    temp = []
    for fold_ in range(3):
        model = load(open(f'../1st_train_2D/tf_efficientnetv2_b0/lgb_model/lgb_fold{fold}_{fold_}', 'rb'))
        temp.append(model)
    model_list_cwt.append(temp)


    model_list_72 = []
    temp = []
    for fold_ in range(3):
        model = load(open(f'../1st_train_1D/one-encoder_72/lgb_model/lgb_fold{fold}_{fold_}', 'rb'))
        temp.append(model)
    model_list_72.append(temp)

    temp = []
    for fold_ in range(3):
        model = load(open(f'../1st_train_1D/two-encoder_72/lgb_model/lgb_fold{fold}_{fold_}', 'rb'))
        temp.append(model)
    model_list_72.append(temp)

    temp = []
    for fold_ in range(3):
        model = load(open(f'../1st_train_1Dtemporal/tf_efficientnet_b0_72/lgb_model/lgb_fold{fold}_{fold_}', 'rb'))
        temp.append(model)
    model_list_72.append(temp)

    temp = []
    for fold_ in range(3):
        model = load(open(f'../1st_train_1Dtemporal/tf_efficientnet_b3_72/lgb_model/lgb_fold{fold}_{fold_}', 'rb'))
        temp.append(model)
    model_list_72.append(temp)

    temp = []
    for fold_ in range(3):
        model = load(open(f'../1st_train_1Dtemporal/efficientvit_b1_72/lgb_model/lgb_fold{fold}_{fold_}', 'rb'))
        temp.append(model)
    model_list_72.append(temp)

    model_lists = []
    model_lists.append(model_list_eeg)
    model_lists.append(model_list_cwt)
    model_lists.append(model_list_72)

    return model_lists


def get_weight():
    weight_list_eeg = []
    weight = np.load(f'../1st_train_1D/one-encoder/output/weight.npy', allow_pickle=True)
    weight_list_eeg.append(weight)

    weight = np.load(f'../1st_train_1D/two-encoder/output/weight.npy', allow_pickle=True)
    weight_list_eeg.append(weight)

    weight = np.load(f'../1st_train_1Dtemporal/tf_efficientnet_b0/output/weight.npy', allow_pickle=True)
    weight_list_eeg.append(weight)

    weight = np.load(f'../1st_train_1Dtemporal/tf_efficientnet_b3/output/weight.npy', allow_pickle=True)
    weight_list_eeg.append(weight)

    weight = np.load(f'../1st_train_1Dtemporal/efficientvit_b1/output/weight.npy', allow_pickle=True)
    weight_list_eeg.append(weight)

    weight = np.load(f'../1st_train_1Dtemporal/tf_efficientnetv2_b0/output/weight.npy', allow_pickle=True)
    weight_list_eeg.append(weight)

    
    weight_list_cwt = []
    weight = np.load(f'../1st_train_2D/tf_efficientnet_b0/output/weight.npy', allow_pickle=True)
    weight_list_cwt.append(weight)

    weight = np.load(f'../1st_train_2D/tf_efficientnet_b3/output/weight.npy', allow_pickle=True)
    weight_list_cwt.append(weight)

    weight = np.load(f'../1st_train_2D/efficientvit_b0/output/weight.npy', allow_pickle=True)
    weight_list_cwt.append(weight)

    weight = np.load(f'../1st_train_2D/efficientvit_b3/output/weight.npy', allow_pickle=True)
    weight_list_cwt.append(weight)

    weight = np.load(f'../1st_train_2D/tf_efficientnetv2_b0/output/weight.npy', allow_pickle=True)
    weight_list_cwt.append(weight)


    weight_list_72 = []
    weight = np.load(f'../1st_train_1D/one-encoder_72/output/weight.npy', allow_pickle=True)
    weight_list_72.append(weight)

    weight = np.load(f'../1st_train_1D/two-encoder_72/output/weight.npy', allow_pickle=True)
    weight_list_72.append(weight)

    weight = np.load(f'../1st_train_1Dtemporal/tf_efficientnet_b0_72/output/weight.npy', allow_pickle=True)
    weight_list_72.append(weight)

    weight = np.load(f'../1st_train_1Dtemporal/tf_efficientnet_b3_72/output/weight.npy', allow_pickle=True)
    weight_list_72.append(weight)

    weight = np.load(f'../1st_train_1Dtemporal/efficientvit_b1_72/output/weight.npy', allow_pickle=True)
    weight_list_72.append(weight)

    weight_lists = []
    weight_lists.append(weight_list_eeg)
    weight_lists.append(weight_list_cwt)
    weight_lists.append(weight_list_72)

    return weight_lists


def predict(X, X_72, nn_model_lists, lgb_model_lists, weight_lists, fold):
    preds = []
    pipeline = MyPipeline(samplerate=500, lowcut=1, highcut=180, wavelet_width=1, n_scales=16, stride=1).to(device)
    for num, (nn_model_list, lgb_model_list, weight_list) in enumerate(zip(nn_model_lists, lgb_model_lists, weight_lists)):
        for nn_model, lgb_model, weight in zip(nn_model_list, lgb_model_list, weight_list):
            with torch.no_grad():
                if num == 2:
                    y_pred, features = nn_model(X_72)
                elif num == 1:
                    y_pred, features = nn_model(pipeline(X))
                else:
                    y_pred, features = nn_model(X)
            nn_pred = nn.functional.softmax(y_pred, dim=1).detach().cpu().numpy()

            lgb_pred = []
            for model in lgb_model:
                proba = model.predict_proba(features.detach().cpu().numpy(), num_iteration=model.best_iteration_)
                lgb_pred.append(proba)
            lgb_pred = np.mean(np.stack(lgb_pred, axis=0), axis=0)

            y_pred = weight[fold] * nn_pred + (1 - weight[fold]) * lgb_pred

            preds.append(y_pred)
    preds = torch.from_numpy(np.stack(preds, axis=0))

    return preds

In [ ]:
target_list = ['subject0', 'subject1', 'subject2', 'subject3', 'subject4']
weight_lists = get_weight()
predict_list = []
for target in target_list:
    print(f'calculating -> {target}')
    data = data_dict[target]
    preds = []
    for fold in range(3):
        dataset = SkateDataset(fold, data, diff_list, use_ch_dict)
        X = dataset.eeg_signals
        X = torch.from_numpy(X).to(device).float()
        X_72 = dataset.eeg_signals_72
        X_72 = torch.from_numpy(X_72).to(device).float()

        nn_model_lists = get_model(fold)
        lgb_model_lists = get_lgb_model(fold)
        y_pred = predict(X, X_72, nn_model_lists, lgb_model_lists, weight_lists, fold)
        preds.append(y_pred)
    preds = torch.argmax(torch.mean(torch.cat(preds, dim=0), dim=0), dim=1)
    for i, pred in enumerate(preds):
        if i < 10:
            number = f'00{i}'
        elif i < 100:
            number = f'0{i}'
        else:
            number = str(i)
        target_name = f'{target}_{number}'
        label = label_dict[pred.item()]
        predict_list.append([target_name, label])

In [ ]:
df = pd.DataFrame(predict_list)
df.to_csv('submit.csv', header=None, index=None)